In [72]:
# from IPython.display import clear_output
# !pip install geopy
# !pip install geopandas
# !pip install basemap
# !pip install folium
# clear_output()

In [102]:
import openpyxl
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import folium
import random
from folium import plugins
import time

In [74]:
def import_excel_data(file_path, sheet_name):
    # Load the workbook and select the desired sheet
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook[sheet_name]

    # Get labels from row 2 (B2 to G2)
    labels = [sheet.cell(row=2, column=col).value for col in range(2, 7)]  # Columns B-G
    # print(labels)
    
    # Check if all labels are present
    if not all(labels):
        raise ValueError("One or more labels in row 2 are missing.")

    # Initialize a dictionary to store the data for each label
    data = {label: [] for label in labels}
    
    # Get data from rows starting at B3 (columns B-G)
    for row in sheet.iter_rows(min_row=3, min_col=2, max_col=7, values_only=True):
        for col_index, value in enumerate(row):
            if value:  # Skip empty cells
                data[labels[col_index]].append(value)
    return data

In [75]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Startup'  # Replace with the name of the sheet

try:
    data_SU = import_excel_data(file_path, sheet_name)
    print(data_SU)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['LightMatter', 'Lightelligence', 'Ayar Labs', 'PsiQuantum', 'Quantum Transistors', 'Q.ANT', 'Lumai ', 'iPRONICS', 'Salience Labs', 'Pixel Photonics', 'Dream Photonics ', 'Akhetonics', 'Celestial AI', 'Xscape Photonics', 'Optelligence', 'Nubis Communications', 'Voyant Photonics', 'Aegiq', 'Nu-quantum', 'Miraex', 'Nordic Quantum Computing Group', 'Orca Computing', 'Photonic Inc', 'Quandela ', 'Quix', 'Sparrow Quantum', 'Lumiphase', 'Enosemi', 'Nexus Photonics', 'Black Semiconductor'], 'Size': [200, 200, '200+', 280, 50, 100, 20, '< 50', '< 50', '< 50', 10, '< 50', '< 200', '< 50', '< 3', '< 50 ', '< 30', '< 50 ', '< 50 ', 50, 'CLOSED 11/2024', '~100', '< 50', '< 50', '< 50', '< 50 ', '< 100 ', '< 50 ', '< 20 ', '<200'], 'Focus': ['Optical MVM / PNN', 'Optical MVM / Networking', 'Optical I/O for AI, Interconnection ', 'Quantum Computing / Optical Interconnect', 'solid-state\xa0quantum\xa0processor', 'Optical MVM', '3D optical AI computing ', 'Microwave Photonics and Netw

In [76]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Large Company'  # Replace with the name of the sheet

try:
    data_LC = import_excel_data(file_path, sheet_name)
    print(data_LC)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['Nvidia Research', 'Google ', 'NEC', 'Intel ', 'Universal Display Corporation', 'OpenAI', 'Apple', 'AMD', 'Cisco', 'Luxtera', 'Infinera ', 'NeoPhotonics', 'Coherent Inc. ', 'Hamamatsu Corporation', 'M Squared\xa0', 'NTT', 'IBM', 'Synopsys Photonic Solutions'], 'Size': ['200+', '300+', '70+', 700, 470, 500, 100], 'Focus': ['Circuit Research Group ', 'Google Quantum AI + Taara @ Google X + Google Platforms', 'Fiber Sensing, Optical Compute, Fiber Telecommunication Optimization', 'Optical input/output (I/O) technology innovation', 'OLED technology, Tangential interest in optical compute', 'quantum technology, biophotonics, and chemical sensing'], 'CEO/Founders/Affliation': ['Tom Gray: Head of CRG', 'Quantum: Hartmut Neven, Taara: Mahesh Krishnaswamy ', 'President: Chris White, ONS Head: Ting Wang, Princeton University', 'Richard Uhlig', 'Sherwin Seligsohn, Stephen Forrest, Princeton University', 'GRAEME MALCOLM OBE, Strathclyde University'], 'Location': ['Santa Clara, Ca

In [77]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Foundries'  # Replace with the name of the sheet

try:
    data_F = import_excel_data(file_path, sheet_name)
    print(data_F)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['Global Foundries', 'AIM Photonics ', 'Advanced Micro Foundry (AMF) ', 'HHI ', 'Smart Photonics', 'LioniX', 'Tower Semiconductor', 'LIGENTEC', 'VTT', 'IMEC', 'CMC Microsystems', 'X-Fab', 'JePPiX'], 'Size': [13000, 500, 130, '300\xa0', '190\xa0', 70, 10000, 100, '2,355\xa0', 5500, '103\xa0', 4500, 20], 'Focus': [' FDX™ FD-SOI, Silicon Photonics, RF SOI, BCD & BCDLite, Bulk CMOS, FinFET 12LP+, SiGe', 'Active/Passive Si, Electronic Interposer, Low-Loss Active Si, Quantum Flex, SiN, Active Interposer, III-V, SiN Visible  ', 'SOI, 800nm SiN on SOI, CoPackaging,  ', 'MPW, InP, RF PIC', 'MPW, InP ', 'SiN TriPleX , Visible, 1550nm, MEMs', '200mm Si, SiN, Darpa LUMOS,  RF, CMOS, MEMS', 'All-Nitride-Core Photonics, ', 'TFLN, Hybrid Integration, Heterogenenous Intergration, High depth Si, Packaging', '200nm CMOS+Si, 300nm CMOS+Si, MPW, Volume, SiN, Packaging, Research', 'Canadian consortium for integrated photonics and packaging', '350nm - 130nm CMOS, RF MEMS,  SIC, GaN, Microfl

In [78]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Defense Contractors'  # Replace with the name of the sheet

try:
    data_DC = import_excel_data(file_path, sheet_name)
    print(data_DC)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['Bascom Hunter', 'Photonic Systems Inc', 'Phase Sensitive Innovations'], 'Size': ['< 100', '< 100', '< 100'], 'Focus': ['Microwave Photonics, Neuromorphioc Photonics', 'Microwave Photonics, Interference Cancellation', 'Microwave Photonics, Diffractive Optics, Phase Arrays, Thin Film LiNo'], 'CEO/Founders/Affliation': ['Paul R. Prucnal , Andy McCandless', 'Charles Cox, Edward Ackerman', 'Dennis Prather, Univ of Delware '], 'Location': ['Baton Rouge, LA and St. Louis, MO', 'Billerica, MA', 'Newark, DE, USA']}


### If a company has multiple locations splits that into new rows and then duplicate the information to the new rows, the deliminator is " and "

In [79]:
keyword = " and "

In [80]:
# search and split
def search_and_split(Locations, keyword):
    results = []
    split_strings = []
    NewLocations = []
    Indx2Dup = []
    j=0
    NewLocations = Locations
    for idx, string in enumerate(Locations):
        # print(idx)
        start_idx = string.find(keyword)
        if start_idx != -1:  # Keyword found
            end_idx = start_idx + len(keyword) - 1
            results.append((idx, start_idx, end_idx))

            # Split the string into two parts: before and after the keyword
            before_keyword = string[:start_idx].strip()
            after_keyword = string[end_idx + 1:].strip()
            split_strings.append((before_keyword, after_keyword))
            
            # Append and replace new split string into original list
            # Conditionals are to look for blank spaces or punctions and not include them 
            threshold = 2
            if len(before_keyword) > threshold and len(after_keyword) > threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = before_keyword
                NewLocations.insert(idx+1, after_keyword)
                #idx is no longer real as New List grows, need to keep track
                Indx2Dup.append(idx-j)
                j+=1
                #When duplicating the rows if there are consecutive number dup from 
                    # row - number of consecutive numbers
                
            if len(before_keyword) <= threshold and len(after_keyword) > threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = after_keyword
                
            if len(before_keyword) > threshold and len(after_keyword) <= threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = before_keyword            

                
    return results, split_strings, NewLocations, Indx2Dup


In [81]:
# Duplicate: 
def duplicate_values_with_repeated_indices(indices, source_list):
    # Create a new list to store the results
    result = []

    for i in range(len(source_list)):
        count = indices.count(i)  # Count occurrences of the index in the indices array
        result.extend([source_list[i]] * (count + 1))  # Original + count of duplicates

    return result

In [82]:
# Testing new search and split function
# strings = [
#     "This is a sample string and it contains and perhaps denver and.",
#     "Another example without the keyword.",
#     "and is a conjunction.",
#     "Nothing here or here",
#     "But maybe here and there"
# ]

# # Search for the keyword "and" and split strings
# keyword = "and" # and is in Netherlands hahaha
# matches, splits, nLocations, Indx = search_and_split(strings, keyword)

# # Print results
# print("Keyword Matches (Start and End Indices):")
# for match in matches:
#     print(f"String index: {match[0]}, Start index: {match[1]}, End index: {match[2]}")

# print("\nSplit Strings:")
# for idx, (before, after) in enumerate(splits):
#     print(f"Original String Index: {matches[idx][0]}")
#     print(f"Before: '{before}'")
#     print(f"After: '{after}'")
    
# # print("\nResult")
# # print(nLocations)

print("\nLocation List:")
print(data_SU["Location"])
print(len(data_SU["Location"]))

matches, splits, nLocations, Indx = search_and_split(data_SU["Location"], keyword)

print("\nNew Location List:")
print(nLocations)
print(len(nLocations))

print("\nIndex to Duplicate")
print(Indx)


Location List:
['Mountain View, CA and Toronto, Canada, and Boston, MA', 'Boston, MA', 'San Jose, CA', 'Palo Alto, CA', 'Tel Aviv, Israel and NYC, US', 'Stuttgart, Germany', 'Oxford, England', 'Valencia, Spain', 'Oxford, England', 'Germany', 'Vancouver, BC, Canada', 'Berlin, Germany', 'Santa Clara, CA, USA', 'New York, NY, USA', 'Austin, USA', 'Murray Hill, New Jersey, USA', 'New York, NY 10018', 'Sheffield, England', 'Cambridge, UK', 'Écublens, Vaud, Switzerland', 'Oslo, Norway ', 'London, UK and Toronto, Canada, and Austin, Tx, USA', 'Vancouver, BC, Canada', 'Massy, Île-de-France', 'Ulm, Germany and Stuttgart Germany, and Amsterdam, Netherlands, and Enschede, Netherlands', 'Copenhagen, Denmark', 'Zurich, Switzerland', 'California,USA', 'Goleta, California, USA', 'Aachen, Germany']
30

New Location List:
['Mountain View, CA', 'Toronto, Canada,', 'Boston, MA', 'Boston, MA', 'San Jose, CA', 'Palo Alto, CA', 'Tel Aviv, Israel', 'NYC, US', 'Stuttgart, Germany', 'Oxford, England', 'Valenc

In [83]:
# print(data_SU.keys())

In [84]:
# # Example usage
# indices = [1, 1, 3, 3, 3, 5]  # Indices for duplication
# source_list = ["a", "b", "c", "d", "e", "f"]

# # Duplicate values based on repeated indices
# result = duplicate_values_with_repeated_indices(indices, source_list)
# print(result)

In [85]:
# Duplicate values at the specified indices for start up tab
data_SU['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_SU['Company Name'])
data_SU['Size'] = duplicate_values_with_repeated_indices(Indx, data_SU['Size'])
data_SU['Focus'] = duplicate_values_with_repeated_indices(Indx, data_SU['Focus'])
data_SU['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_SU['CEO/Founders/Affliation'])


In [86]:
# Same for LC, F, and DC
matches, splits, nLocations, Indx = search_and_split(data_LC["Location"], keyword)
data_LC['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_LC['Company Name'])
data_LC['Size'] = duplicate_values_with_repeated_indices(Indx, data_LC['Size'])
data_LC['Focus'] = duplicate_values_with_repeated_indices(Indx, data_LC['Focus'])
data_LC['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_LC['CEO/Founders/Affliation'])

matches, splits, nLocations, Indx = search_and_split(data_F["Location"], keyword)
data_F['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_F['Company Name'])
data_F['Size'] = duplicate_values_with_repeated_indices(Indx, data_F['Size'])
data_F['Focus'] = duplicate_values_with_repeated_indices(Indx, data_F['Focus'])
data_F['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_F['CEO/Founders/Affliation'])

matches, splits, nLocations, Indx = search_and_split(data_DC["Location"], keyword)
data_DC['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_DC['Company Name'])
data_DC['Size'] = duplicate_values_with_repeated_indices(Indx, data_DC['Size'])
data_DC['Focus'] = duplicate_values_with_repeated_indices(Indx, data_DC['Focus'])
data_DC['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_DC['CEO/Founders/Affliation'])


### Now I have corrected list for companies with multiple location, we convert rough individual locations into coordinates and make a new column

In [114]:
# Convert Rough Location to longitude and latitude

def get_coordinates_with_duplicates(locations, w_Debug=False):
    # Initialize the geolocator
    geolocator = Nominatim(user_agent="location_to_coordinates")
    cache = {}  # Cache to store already-queried locations
    results = []  # Final list of results
    slow_down = 0.5
    for location in locations:
        tries = 3
        time.sleep(slow_down)
        if location in cache:
            # Use cached result if available
            results.append(cache[location])
        else:
            try:
                # Geocode the location
                if w_Debug == True:
                    print(location)
                for i in range(tries):
                    try:
                        loc = geolocator.geocode(location)
                    except GeocoderTimedOut as e:
                        if i < tries - 1: # i is zero indexed
                            time.sleep(1)
                            print(Location, "failed, trying again")
                            continue
                        else:
                            raise
                if loc:
                    coordinates = (loc.latitude, loc.longitude)
                else:
                    coordinates = None
                    print('Could find coordinates for ',location, '.')
                    print('Please change the location name within the database.\n')
                # Cache the result and append to results
                cache[location] = coordinates
                results.append(coordinates)
            except GeocoderTimedOut:
                # Handle timeout gracefully
                print("Failed at providing ", location, " at least ", tries, " number of times.")
                results.append(None)

    return results

In [93]:
print(data_SU["Location"])

['Mountain View, CA', 'Toronto, Canada,', 'Boston, MA', 'Boston, MA', 'San Jose, CA', 'Palo Alto, CA', 'Tel Aviv, Israel', 'NYC, US', 'Stuttgart, Germany', 'Oxford, England', 'Valencia, Spain', 'Oxford, England', 'Germany', 'Vancouver, BC, Canada', 'Berlin, Germany', 'Santa Clara, CA, USA', 'New York, NY, USA', 'Austin, USA', 'Murray Hill, New Jersey, USA', 'New York, NY 10018', 'Sheffield, England', 'Cambridge, UK', 'Écublens, Vaud, Switzerland', 'Oslo, Norway ', 'London, UK', 'Toronto, Canada,', 'Austin, Tx, USA', 'Vancouver, BC, Canada', 'Massy, Île-de-France', 'Ulm, Germany', 'Stuttgart Germany,', 'Amsterdam, Netherlands,', 'Enschede, Netherlands', 'Copenhagen, Denmark', 'Zurich, Switzerland', 'California,USA', 'Goleta, California, USA', 'Aachen, Germany']


### Check here if any rough addresses aren't able to be converted

In [20]:
# Get coordinates from location list 
# Sometimes fails, add true as second input to function and rerun
# Could be just timing out, could be a bad location in the spread sheet
# if timing out, you can add more time out to slow down within the function
coordinates_SU = get_coordinates_with_duplicates(data_SU["Location"])

In [21]:
coordinates_LC = get_coordinates_with_duplicates(data_LC["Location"])

In [ ]:
coordinates_F = get_coordinates_with_duplicates(data_F["Location"])

In [105]:
coordinates_DC = get_coordinates_with_duplicates(data_DC["Location"])

In [106]:
# # Example list of locations
# locations = [
#     "Boston, MA, USA",
#     "New York, NY, USA",
#     "Boston, MA, USA",  # Duplicate
#     "San Francisco, CA, USA",
#     "New York, NY, USA"  # Duplicate
# ]

# # Get coordinates
# coordinates = get_coordinates_with_duplicates(locations)

# # Print results
# for location, coord in zip(locations, coordinates):
#     if coord:
#         print(f"{location}: Latitude = {coord[0]}, Longitude = {coord[1]}")
#     else:
#         print(f"{location}: Coordinates not found.")

### Cost-of-living 

In [107]:
# Maybe scrape from Numbeo
# Kinda difficult to find an easy dataset for this 

### Plot coordinates on map

In [108]:
# https://python-visualization.github.io/folium/latest/index.html

In [109]:
def generate_random_pair():
    pairs = []
    Variation_Magnitiude = 0.02
    Exclude_Zone = 0.01
    while True:
        num1 = random.uniform(-Variation_Magnitiude, Variation_Magnitiude)
        num2 = random.uniform(-Variation_Magnitiude, Variation_Magnitiude)
        if not (-Exclude_Zone <= num1 <= Exclude_Zone or -Exclude_Zone <= num2 <= Exclude_Zone):
            pairs = [num1, num2]
            break
    return pairs

# # Example usage
# random_pair = generate_random_pair()
# print(f"Generated random pair: {random_pair}")

In [110]:
# # Example using folium 
# m = folium.Map(location=[41,29])

# # folium.Circle(
# #     radius=5000,
# #     location=[41,29],
# #     color='crimson',
# #     fill=False,).add_to(m)

# # folium.CircleMarker(location=(41,29),radius=100, fill_color='red').add_to(m)

# folium.Marker(location=[41,29], popup =  'Sakarya').add_to(m)
# new_pair = generate_random_pair()
# folium.Marker(location=[41+new_pair[0],29+new_pair[1]], popup =  'Sakarya').add_to(m)

# minimap = plugins.MiniMap()
# m.add_child(minimap)

# m


In [111]:
# Plotting SUs on World Map
# Add a slight random variation to coordinates to make markers in high density areas (the bay) more legible 

#Init map with starting location at SF
SF = [37.7749, -122.4194]
USA = [40, -98]
Zoom = 4
CompanyMap = folium.Map(USA, tiles="cartodb positron", zoom_start=Zoom)
color_SU = "darkblue"
color_LC = "red"
color_F = "green"
color_DC = "purple"

for new_cords, new_CompName, new_Size, new_Focus, new_Founders \
    in  zip(coordinates_SU, data_SU['Company Name'], data_SU['Size'], \
            data_SU['Focus'],data_SU['CEO/Founders/Affliation']):  
    variation = generate_random_pair()
    folium.Marker(location=[a + b for a, b in zip(new_cords, variation)], \
                  tooltip=new_CompName,
                  popup = new_CompName+'\n Size: ' + str(new_Size) + \
                 '\n Focus: ' + new_Focus + '\n Key Members and Affliations: ' \
                 + new_Founders, \
                 icon=folium.Icon(icon="info-sign", icon_color ="white", \
                                  color = color_SU)).add_to(CompanyMap)

for new_cords, new_CompName, new_Size, new_Focus, new_Founders \
    in  zip(coordinates_LC, data_LC['Company Name'], data_LC['Size'], \
            data_LC['Focus'],data_LC['CEO/Founders/Affliation']):  
    variation = generate_random_pair()
    folium.Marker(location=[a + b for a, b in zip(new_cords, variation)], \
                  tooltip=new_CompName,
                  popup = new_CompName+'\n Size: ' + str(new_Size) + \
                 '\n Focus: ' + new_Focus + '\n Key Members and Affliations: ' \
                 + new_Founders, \
                 icon=folium.Icon(icon="info-sign", icon_color ="white", color = color_LC)).add_to(CompanyMap)

for new_cords, new_CompName, new_Size, new_Focus, new_Founders \
    in  zip(coordinates_F, data_F['Company Name'], data_F['Size'], \
            data_F['Focus'],data_F['CEO/Founders/Affliation']):  
    variation = generate_random_pair()
    folium.Marker(location=[a + b for a, b in zip(new_cords, variation)], \
                  tooltip=new_CompName,
                  popup = new_CompName+'\n Size: ' + str(new_Size) + \
                 '\n Focus: ' + new_Focus + '\n Key Members and Affliations: ' \
                 + new_Founders, \
                 icon=folium.Icon(icon="info-sign", icon_color ="white", color = color_F)).add_to(CompanyMap)

for new_cords, new_CompName, new_Size, new_Focus, new_Founders \
    in  zip(coordinates_DC, data_DC['Company Name'], data_DC['Size'], \
            data_DC['Focus'],data_DC['CEO/Founders/Affliation']):  
    variation = generate_random_pair()
    folium.Marker(location=[a + b for a, b in zip(new_cords, variation)], \
                  tooltip=new_CompName,
                  popup = new_CompName+'\n Size: ' + str(new_Size) + \
                 '\n Focus: ' + new_Focus + '\n Key Members and Affliations: ' \
                 + new_Founders, \
                 icon=folium.Icon(icon="info-sign", icon_color ="white", color = color_DC)).add_to(CompanyMap)

    
minimap = plugins.MiniMap()
CompanyMap.add_child(minimap)

CompanyMap

# ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
#  |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
#  |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
#  |           'gray', 'black', 'lightgray']

In [112]:

# help(folium.Icon())